In [1]:
# This file is part of pydidas.
#
# Copyright 2023 - 2024, Helmholtz-Zentrum Hereon
# SPDX-License-Identifier: GPL-3.0-only
#
# pydidas is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License version 3 as
# published by the Free Software Foundation.
#
# Pydidas is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Pydidas. If not, see <http://www.gnu.org/licenses/>.

"""
The dataset module includes the Dataset subclasses of numpy.ndarray with additional
embedded metadata.
"""

__author__ = "Gudrun Lotze"
__copyright__ = "Copyright 2024, Helmholtz-Zentrum Hereon"
__license__ = "GPL-3.0-only"
__maintainer__ = "Malte Storm, Gudrun Lotze"
__status__ = "Development"

In [3]:
import os
import h5py as h5
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LogNorm
import matplotlib.cm as cm
from scipy.optimize import curve_fit
from pydidas.core import Dataset
from pydidas.data_io import import_data
%matplotlib inline

from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components


import importlib

fname = '/Users/lotzegud/pydidas/pydidas_plugins/proc_plugins/sin2chi.py'

spec = importlib.util.spec_from_file_location('sin2psi', fname)
s2c_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(s2c_module)


#vielleicht bessere Namen fuer die Funktionen benutzen, gerne auch ausfuehrlich

